In [1]:
import pandas as pd
from gurobipy import *
import matplotlib.pyplot as plt
import math
import numpy as np
import datetime as dt

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 500)

In [3]:
booking = pd.read_excel("Data/Booking_new.xlsx")
bd_zones = pd.read_excel("Data/BreakDownZones.xlsx")
dz_to_bd = pd.read_excel("Data/DistanceMatrixDropZoneToBreakDownZone.xlsx")
drop_zones = pd.read_excel("Data/DropZones.xlsx")
bu_zones = pd.read_excel("Data/Outbound.xlsx", sheet_name='BU Zones')
bu_zones_workstations = pd.read_excel("Data/Outbound.xlsx", sheet_name='WorkStation')
bu_zones_to_flight = pd.read_excel("Data/Outbound.xlsx", sheet_name='FlightNumber-BUZone')
flight_default_processing = pd.read_excel("Data/Outbound.xlsx", sheet_name='DefaultProcessingTime')
flight_pre_processing = pd.read_excel("Data/Outbound.xlsx", sheet_name='Pre-ProcessingBufferTime')

FileNotFoundError: [Errno 2] No such file or directory: 'Data/Booking_new.xlsx'

In [ ]:
flight_pre_processing['Weekday'].unique()

In [ ]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [ ]:
all_days_pre = flight_pre_processing[flight_pre_processing.Weekday=='*']

In [ ]:
all_days_pro = []
for d in days:
    for index, row in all_days_pre.iterrows():
        all_days_pro.append((row['FlightNumber'], row['Weekday'], row['PreProcessingBufferTime'], d))

all_days_pro1 = pd.DataFrame(all_days_pro, columns=('FlightNumber', 'Weekday', 'PreProcessingBufferTime', 'day'))
all_days_pro1 = all_days_pro1.sort_values(by=['FlightNumber'])

In [ ]:
sunday_days_pre = flight_pre_processing[flight_pre_processing.Weekday==7]

In [ ]:
sunday_days_pro = []
for d in ['Sunday']:
    for index, row in sunday_days_pre.iterrows():
        sunday_days_pro.append((row['FlightNumber'], row['Weekday'], row['PreProcessingBufferTime'], d))

sunday_days_pro1 = pd.DataFrame(sunday_days_pro, columns=('FlightNumber', 'Weekday', 'PreProcessingBufferTime', 'day'))
sunday_days_pro1 = sunday_days_pro1.sort_values(by=['FlightNumber'])

In [ ]:
saturday_days_pre = flight_pre_processing[flight_pre_processing.Weekday==6]

In [ ]:
saturday_days_pro = []
for d in ['Saturday']:
    for index, row in saturday_days_pre.iterrows():
        saturday_days_pro.append((row['FlightNumber'], row['Weekday'], row['PreProcessingBufferTime'], d))

saturday_days_pro1 = pd.DataFrame(saturday_days_pro, columns=('FlightNumber', 'Weekday', 'PreProcessingBufferTime', 'day'))
saturday_days_pro1 = saturday_days_pro1.sort_values(by=['FlightNumber'])

In [ ]:
flight_pre_processing_all = all_days_pro1.append(sunday_days_pro1)
flight_pre_processing_all = flight_pre_processing_all.append(saturday_days_pro1)

In [ ]:
flight_pre_processing_all[['Pre_h','Pre_m','Pre_s']] = pd.DataFrame([(x.hour, x.minute, x.second) for x in flight_pre_processing_all['PreProcessingBufferTime']])

In [ ]:
flight_pre_processing_all['flight_pre_id'] = flight_pre_processing_all['FlightNumber'].astype(str) + "_" + flight_pre_processing_all['day']

In [ ]:
flight_pre_processing_all.head()

In [ ]:
flight_default_processing[['default_h','default_m','default_s']] = pd.DataFrame([(x.hour, x.minute, x.second) for x in flight_default_processing['DefaultProcessingTime']])

In [ ]:
bu_zones_to_flight[['transport_dist_h','transport_dist_m','transport_dist_s']] = pd.DataFrame([(x.hour, x.minute, x.second) for x in bu_zones_to_flight['TransportationDistance']])

In [ ]:
bu_zones_to_flight.shape

In [ ]:
bu_zones_to_flight = bu_zones_to_flight.merge(flight_default_processing, on=['FlightNumber'])

In [ ]:
bu_zones_to_flight.head()

In [ ]:
dz = list(drop_zones['Name'].unique())
break_down_zones = list(bd_zones['Name'].unique())

In [ ]:
dz_to_bd[['h','m','s']] = pd.DataFrame([(x.hour, x.minute, x.second) for x in dz_to_bd['TransportDuration']])
bd_zones[['handling_h','handling_m','handling_s']] = pd.DataFrame([(x.hour, x.minute, x.second) for x in bd_zones['HandlingTimePerULD']])
bd_zones[['transport_WH_h','transport_WH_m','transport_WH_s']] = pd.DataFrame([(x.hour, x.minute, x.second) for x in bd_zones['TransportationTimeToWH']])

In [ ]:
booking['key_shipment_id_arrival_date'] = booking['Shipment ID'].astype(str) + booking['ShipmentArrivalDateUTC'].astype(str)
booking = booking.drop_duplicates('key_shipment_id_arrival_date')

In [ ]:
booking = booking[booking.TotalWeight<=400]

In [ ]:
#####################################################################################
#Handling multiple ULDs having same number arriving less than 4 hours

In [ ]:
uld_number_times = booking[['ShipmentArrivalDateUTC', 'ArrivalULDNumber']]
uld_number_times['id'] = booking['ShipmentArrivalDateUTC'].astype(str) + '_' + booking['ArrivalULDNumber'].astype(str)
uld_number_times = uld_number_times.drop_duplicates('id')
uld_number_times.shape

In [ ]:
uld_number_times['count_uld_number'] = 1

In [ ]:
unique_uld_count_by_number = uld_number_times.groupby(['ArrivalULDNumber']).agg({"count_uld_number":"count"}).rename(columns={"count_uld_number":"Count"}).reset_index()
multiple_uld_numbers = unique_uld_count_by_number[unique_uld_count_by_number.Count>1]

In [ ]:
multiple_uld_number_and_times = uld_number_times[uld_number_times['ArrivalULDNumber'].isin((multiple_uld_numbers['ArrivalULDNumber'].unique()))]
multiple_uld_number_and_times = multiple_uld_number_and_times.sort_values(by=['ArrivalULDNumber', 'ShipmentArrivalDateUTC'])
multiple_uld_number_and_times['arrival_time'] = pd.to_datetime(multiple_uld_number_and_times['ShipmentArrivalDateUTC'])
multiple_uld_number_and_times['diff'] = multiple_uld_number_and_times.sort_values(by=['ArrivalULDNumber', 'arrival_time']).groupby('ArrivalULDNumber')['arrival_time'].diff()
multiple_ULD_arriving_below_4_hours = list(multiple_uld_number_and_times[multiple_uld_number_and_times['diff'].notnull() & (multiple_uld_number_and_times['diff'] < pd.Timedelta(4, unit='h'))]['ArrivalULDNumber'].unique())
booking = booking[~booking['ArrivalULDNumber'].isin(multiple_ULD_arriving_below_4_hours)]

In [ ]:
#######################################################################################

In [ ]:
# Filtering shipments arriving after departure
booking['Shipment_arrival_date_time'] = pd.to_datetime(booking['ShipmentArrivalDateUTC'])
booking = booking[booking.ShipmentDepartureDateUTC != '2018-10-24T26:20:00']
booking['Shipment_departure_date_time'] = pd.to_datetime(booking['ShipmentDepartureDateUTC'])
booking['Shipment_duration'] = booking['Shipment_departure_date_time']-booking['Shipment_arrival_date_time']
booking = booking[booking.Shipment_duration>pd.Timedelta(0)]

In [ ]:
#######################################################################################

In [ ]:
booking['count_shipments'] = 1
multple_dropzone_check = booking.pivot_table(index=['ArrivalULDNumber', 'Shipment_arrival_date_time'], columns='DropZone', values= 'count_shipments').reset_index()
multple_dropzone_check = multple_dropzone_check.fillna(0)
multple_dropzone_check['Two_different_dropzones_check'] = multple_dropzone_check['DZ NML-1'] + multple_dropzone_check['DZ NRML-1'] + multple_dropzone_check['DZ NRML-2']
unique_dropzone_check = multple_dropzone_check[multple_dropzone_check.Two_different_dropzones_check==1.0]
two_drop_zones = multple_dropzone_check[multple_dropzone_check.Two_different_dropzones_check>1.0]

In [ ]:
two_drop_zones['unique_id'] = two_drop_zones['ArrivalULDNumber'].astype(str) + '_' + (pd.to_timedelta(two_drop_zones['Shipment_arrival_date_time']).astype('timedelta64[m]').astype(int).astype(str))

In [ ]:
dz_to_bd['DropZoneName_filter'] = dz_to_bd['DropZoneName'].apply(lambda x: x.split('-')[0]).str[3:]
dz_to_bd['bd_type1'] = dz_to_bd['BreakDownZoneName'].apply(lambda x: x.split(' ')[1])
dz_to_bd.loc[dz_to_bd['BreakDownZoneName'] == 'B BD NRML-1', 'bd_type1'] = "NRML-1"
dz_to_bd.loc[dz_to_bd['BreakDownZoneName'] == 'B BD NRML-2', 'bd_type1'] = "NRML-2"
dz_to_bd['bd_type'] =  dz_to_bd['bd_type1'].apply(lambda x: x.split('-')[0])
dz_to_bd['check'] = np.where(dz_to_bd['DropZoneName_filter']==dz_to_bd['bd_type'], 1, 0)
dz_to_bd = dz_to_bd[dz_to_bd['check']==1]
dz_to_bd = dz_to_bd[['BreakDownZoneName', 'DropZoneName', 'TransportDuration', 'h', 'm', 's']]

In [ ]:
dz_ship_data = booking[['DropZone','ArrivalULDNumber', 'ShipmentArrivalDateUTC']]
dz_ship_data['unique_key'] =  dz_ship_data['DropZone'].astype(str) + '_' + dz_ship_data['ArrivalULDNumber'].astype(str) + '_' + dz_ship_data['ShipmentArrivalDateUTC'].astype(str)
dz_ship_data = dz_ship_data.drop_duplicates('unique_key')
dz_ship_data = dz_ship_data[['DropZone', 'ArrivalULDNumber', 'ShipmentArrivalDateUTC']]

In [ ]:
uld_bd_data = pd.merge(dz_ship_data, dz_to_bd, left_on='DropZone', right_on='DropZoneName')

In [ ]:
uld_bd_data['key'] = uld_bd_data['DropZone'].astype(str) + '_' + uld_bd_data['ArrivalULDNumber'].astype(str) + '_' + uld_bd_data['ShipmentArrivalDateUTC'].astype(str) + '_' + uld_bd_data['BreakDownZoneName'].astype(str)
uld_bd_data = uld_bd_data.drop_duplicates('key')

In [ ]:
uld_bd_data['Shipment_arrival_date_time'] = pd.to_datetime(uld_bd_data['ShipmentArrivalDateUTC'])

In [ ]:
#Getting BU workstations count
bu_workstation_count = bu_zones_workstations.groupby(['Name']).count().reset_index()
bu_workstation_count.columns = ['Name', 'Count']

In [ ]:
bu_zones_data = pd.merge(bu_zones_to_flight, bu_zones, left_on='BU Zone', right_on='Name')
bu_zone_all = pd.merge(bu_zones_data, bu_workstation_count, left_on='BU Zone', right_on='Name')

In [ ]:
bu_zone_all[['handling_ULD_h','handling_ULD_m','handling_ULD_s']] = pd.DataFrame([(x.hour, x.minute, x.second) for x in bu_zone_all['HandlingTimePerULD']])
bu_zone_all[['transport_WH_h','transport_WH_m','transport_WH_s']] = pd.DataFrame([(x.hour, x.minute, x.second) for x in bu_zone_all['TransportationTimeToWH']])

In [ ]:
booking.head()

In [ ]:
#Creating booking_new variable for future use
booking['build_up_arrival_time'] =  booking['Shipment_arrival_date_time'] #+ np.timedelta64(2, 'h')
booking['build_up_finish_time'] =  booking['Shipment_departure_date_time'] #- np.timedelta64(1, 'h')
booking_new = booking[['Shipment ID','ProductName', 'Priority', 'DropZone', 'ArrivalULDNumber', 'Pieces', 'Weight', 'TotalWeight',
         'FlightNumberDeparture', 'key_shipment_id_arrival_date', 'Shipment_arrival_date_time', 'Shipment_departure_date_time',
         'Shipment_duration', 'build_up_arrival_time', 'build_up_finish_time']]

In [ ]:
booking_new = pd.merge(booking_new, bu_zone_all, left_on='FlightNumberDeparture', right_on='FlightNumber')

In [ ]:
booking_new[['ULD_handling_h','ULD_handling_m','ULD_handling_s']] = pd.DataFrame([(x.hour, x.minute, x.second) for x in booking_new['HandlingTimePerULD']])
booking_new['arrival_date'] = booking_new['Shipment_arrival_date_time'].dt.normalize()
booking_new['departure_date'] = booking_new['Shipment_departure_date_time'].dt.normalize()

In [ ]:
# DataFrame used in  BD zone
#sample_shipments = booking_new.iloc[98:118,:]
# sample_shipments = booking_new[booking_new['ArrivalULDNumber'].isin(list([34842, 34222, 92870, 21231, 49815, 51228, 59243]))]
# sample_shipments = booking_new[booking_new['ArrivalULDNumber'].isin(list([34842, 34222, 92870]))]
# sample_shipments = booking_new[booking_new['ArrivalULDNumber'].isin(list([34842, 34222,92870, 21231, 49815, 59243]))]
# sample_shipments = booking_new[booking_new['ArrivalULDNumber'].isin(list([51228]))]
sample_shipments = booking_new

In [ ]:
sample_shipments['Shipment_duration_minutes'] = (sample_shipments['Shipment_duration'] / np.timedelta64(1, 'm')).astype(int)
sample_shipments['Shipment_arrival_time_minutes'] = pd.to_timedelta(sample_shipments['Shipment_arrival_date_time']).astype('timedelta64[m]').astype(int)
sample_shipments['Shipment_departure_time_minutes'] = pd.to_timedelta(sample_shipments['Shipment_departure_date_time']).astype('timedelta64[m]').astype(int)

In [ ]:
new = sample_shipments[['DropZone','ArrivalULDNumber', 'Shipment_arrival_date_time', 'Shipment_duration_minutes','Shipment_arrival_time_minutes', 'Shipment_departure_time_minutes']]
df_merge= pd.merge(uld_bd_data, new, on=['DropZone','ArrivalULDNumber', 'Shipment_arrival_date_time'])
df_merge = df_merge.drop_duplicates('key')
bd_capacity = bd_zones[['Name','NumberOfWorkstations', 'handling_m']]
df_merge1 = df_merge.merge(bd_capacity, left_on='BreakDownZoneName', right_on='Name')

In [ ]:
#BD process needed to to be completed with in 60 minutes
df_merge1['one_hour_duration'] = 60

In [ ]:
df_merge1['unique_id'] = df_merge1['ArrivalULDNumber'].astype(str) + '_' + df_merge1['Shipment_arrival_time_minutes'].astype(str)

In [ ]:
two_drop_zones_ULD_Numbers = list(two_drop_zones['unique_id'].unique())
#two_drop_zones_ULD_Numbers = list(two_drop_zones[two_drop_zones['ArrivalULDNumber'].isin([34842, 34222, 92870, 21231, 49815, 51228, 59243])]['unique_id'].unique())

In [ ]:
uld_uniqueid_list = list(df_merge1['unique_id'].unique())

In [ ]:
ArrivalULDNumber_list = list(df_merge1['ArrivalULDNumber'].unique())
BreakDownZoneName_list = list(df_merge1['BreakDownZoneName'].unique())

In [ ]:
#############################################################################################################
##################################################################################################################
###############################################################################################################################################

In [ ]:
sample_shipments['day_name'] = sample_shipments['departure_date'].apply(lambda x: dt.datetime.strftime(x, '%A'))

In [ ]:
sample_shipments['flight_pre_id'] = sample_shipments['FlightNumber'].astype(str) + '_' + sample_shipments['day_name']

In [ ]:
sample_shipments = sample_shipments.merge(flight_pre_processing_all, on=['flight_pre_id'] , how='left')

In [ ]:
sample_shipments = sample_shipments.fillna(0)

In [ ]:
sample_shipments['Pre_m'].unique()

In [ ]:
sample_shipments['FlightNumber_y'] = sample_shipments['FlightNumber_y'].astype('int64')
sample_shipments['Pre_h'] = sample_shipments['Pre_h'].astype('int64')
sample_shipments['Pre_m'] = sample_shipments['Pre_m'].astype('int64')
sample_shipments['Pre_s'] = sample_shipments['Pre_s'].astype('int64')

In [ ]:
sample_shipments['new_departure_time'] = sample_shipments['Shipment_departure_time_minutes'] - sample_shipments['transport_dist_m'] - sample_shipments['default_m'] - sample_shipments['Pre_m']

In [ ]:
sample_shipments['Shipment_departure_time_minutes'] = sample_shipments['new_departure_time']

In [ ]:
booking_new_BU = sample_shipments

In [ ]:
booking_new_BU['shipment_ID'] = range(1, len(booking_new_BU) + 1)
shipment_id_list = list(booking_new_BU['shipment_ID'].unique())

In [ ]:
booking_new_BU.head()

In [ ]:
#Data used for BU process 'sample_shipments_bu'
sample_shipments_bu = booking_new_BU

In [ ]:
sample_shipments_bu['Shipment_departure_time_minutes'].unique()

In [ ]:
sample_shipments_bu['unique_id'] = sample_shipments_bu['ArrivalULDNumber'].astype(str) + '_' + sample_shipments_bu['Shipment_arrival_time_minutes'].astype(str)

In [ ]:
ArrivalULDNumber_list = list(sample_shipments_bu['ArrivalULDNumber'].unique())
filghts_list = list(sample_shipments_bu['FlightNumberDeparture'].unique())
bu_zones_list = list(sample_shipments_bu['Name_x'].unique())

In [ ]:
##################################################################################
#####################################################################################
#######################################################################################

In [ ]:
# Full Model

In [ ]:
#import datetime
print(dt.datetime.now())

In [ ]:
model = Model('Full model')

In [ ]:
#Discretisation has been done based on each BD zone handling times
x = {}
for i in uld_uniqueid_list:
    BreakDownZoneName_list1 = list(df_merge1[df_merge1.unique_id==i]['BreakDownZoneName'])
    #print(BreakDownZoneName_list1)
    for j in BreakDownZoneName_list1:
        #print(int(i),str(j))
        new_check = df_merge1[(df_merge1.unique_id==i) & (df_merge1.BreakDownZoneName==j)]
        min_time = min(list(df_merge1[df_merge1.BreakDownZoneName==j]['Shipment_arrival_time_minutes'].unique()))
        arr = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
            #print(arr,"--",i)
        shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
        handling_minutes = list(new_check['handling_m'].unique())[0]
            #print(shipment_arrival_time_minutes_loop, handling_minutes)
        t_start = (min_time + (math.ceil((arr-min_time)/handling_minutes)*handling_minutes))
            #print(arr,"-----",t_start)
        #if(i == '51228_25671360'):   
            #print(t_start)
        for k in range(t_start, t_start+shipment_arrival_duation_minutes_loop, handling_minutes):
            x[i,j,k] = model.addVar(vtype=GRB.BINARY)

In [ ]:
z = {}
for i in ArrivalULDNumber_list:
    shipment_date_list = list(sample_shipments_bu[sample_shipments_bu.ArrivalULDNumber==i]['departure_date'].unique())
    for d in shipment_date_list:
        shipment_list = list(sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.departure_date==d)]['shipment_ID'])
        #print(shipment_list)
        for l in shipment_list:
        #print(int(i),str(j))
            number_of_ws = sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.shipment_ID==l)]['Count'].unique()[0]
            shipment_departure_time_minutes_loop = sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.shipment_ID==l)]['Shipment_departure_time_minutes'].unique()[0]
            for m in range(1,number_of_ws+1):
                for t in range(shipment_departure_time_minutes_loop-60, shipment_departure_time_minutes_loop):
                    z[l,m,t] = model.addVar(vtype=GRB.BINARY)

In [ ]:
q = {}
for k in filghts_list:
    shipment_date_list = list(sample_shipments_bu[sample_shipments_bu.FlightNumberDeparture==k]['departure_date'].unique())
    for d in shipment_date_list:
        ws_list = sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k) & (sample_shipments_bu.departure_date==d)]['Count'].unique()[0]
        shipment_departure_time_minutes_loop = sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k) & (sample_shipments_bu.departure_date==d)]['Shipment_departure_time_minutes'].unique()[0]
    #print(shipment_list)
        for m in range(1, ws_list+1):
        #print(int(i),str(j))
            for t in range(shipment_departure_time_minutes_loop-60, shipment_departure_time_minutes_loop):
                q[k,m,t] = model.addVar(vtype=GRB.BINARY)

In [ ]:
#BD Constraints

In [ ]:
# #Handling time of each bd zone take into account for the quicksum
for i in (list(set(uld_uniqueid_list) - set(two_drop_zones_ULD_Numbers))):
    BreakDownZoneName_list1 = list(df_merge1[df_merge1.unique_id==i]['BreakDownZoneName'])
    new_check = df_merge1[(df_merge1.unique_id==i)]
    arr = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
    shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
    #min_time = min(list(df_merge1[df_merge1.BreakDownZoneName.isin(BreakDownZoneName_list1)]['Shipment_arrival_time_minutes'].unique()))
    #t_start = shipment_arrival_time_minutes_loop + (handling_minutes - (shipment_arrival_time_minutes_loop%min_time))
    if(len(BreakDownZoneName_list1)!=0):
        model.addConstr(quicksum(x[i,b,c] for b in BreakDownZoneName_list1 for c in range((min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((arr-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))), (min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((arr-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))) +shipment_arrival_duation_minutes_loop, df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m'])) == 1)

In [ ]:
# #Handling time of each bd zone take into account for the quicksum
for i in two_drop_zones_ULD_Numbers:
    #print(i)
    BreakDownZoneName_list1 = list(df_merge1[df_merge1.unique_id==i]['BreakDownZoneName'])
    bd_NML_list = [k for k in BreakDownZoneName_list1 if 'NML' in k]
    bd_NRML_list = [k for k in BreakDownZoneName_list1 if 'NRML' in k]
    #print(BreakDownZoneName_list1)
    #print(bd_NML_list)
    #print(bd_NRML_list)
    new_check = df_merge1[(df_merge1.unique_id==i)]
    arr = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
    #print(arr)
    shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
    #min_time = min(list(df_merge1[df_merge1.BreakDownZoneName.isin(BreakDownZoneName_list1)]['Shipment_arrival_time_minutes'].unique()))
    #t_start = shipment_arrival_time_minutes_loop + (handling_minutes - (shipment_arrival_time_minutes_loop%min_time))
    if(len(bd_NML_list)!=0):
        model.addConstr(quicksum(x[i,b,c] for b in bd_NML_list for c in range((min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((arr-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))), (min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((arr-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))) +shipment_arrival_duation_minutes_loop, df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m'])) == 1)
    if(len(bd_NRML_list)!=0):
        model.addConstr(quicksum(x[i,b,c] for b in bd_NRML_list for c in range((min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((arr-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))), (min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((arr-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))) +shipment_arrival_duation_minutes_loop, df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m'])) == 1)

In [ ]:
#Precedence of NML before NRML
for i in two_drop_zones_ULD_Numbers:
    #print(i)
    BreakDownZoneName_list1 = list(df_merge1[df_merge1.unique_id==i]['BreakDownZoneName'])
    bd_NML_list = [k for k in BreakDownZoneName_list1 if 'NML' in k]
    bd_NRML_list = [k for k in BreakDownZoneName_list1 if 'NRML' in k]
    #print(BreakDownZoneName_list1)
    #print(bd_NML_list)
    #print(bd_NRML_list)
    new_check = df_merge1[(df_merge1.unique_id==i)]
    arr = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
    #print(arr)
    shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
    #min_time = min(list(df_merge1[df_merge1.BreakDownZoneName.isin(BreakDownZoneName_list1)]['Shipment_arrival_time_minutes'].unique()))
    #t_start = shipment_arrival_time_minutes_loop + (handling_minutes - (shipment_arrival_time_minutes_loop%min_time))
    if((len(bd_NML_list)!=0) & (len(bd_NRML_list)!=0)):
        model.addConstr(quicksum((c+(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*x[i,b,c] for b in bd_NML_list for c in range((min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((arr-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))), (min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((arr-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))) +shipment_arrival_duation_minutes_loop, df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m'])) <= quicksum(c*x[i,b,c] for b in bd_NRML_list for c in range((min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((arr-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))), (min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((arr-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))) +shipment_arrival_duation_minutes_loop, df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m'])))

In [ ]:
#BD Assgnment Time constraint
for i in (list(set(uld_uniqueid_list) - set(two_drop_zones_ULD_Numbers))):
    #for j in BreakDownZoneName_list:
        #print(int(i),str(j))
    BreakDownZoneName_list1 = list(df_merge1[df_merge1.unique_id==i]['BreakDownZoneName'])
    new_check = df_merge1[(df_merge1.unique_id==i)]
    arr = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
    shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
    #arrival_time_each_uld = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
    transport_time_each_bd_zone = list(new_check['m'].unique())[0]
    if(len(BreakDownZoneName_list1)!=0):
        model.addConstr(quicksum((c-df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['m'])*x[i,b,c] for b in BreakDownZoneName_list1 for c in range((min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((arr-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))), (min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((arr-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))) +shipment_arrival_duation_minutes_loop, df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m'])) >= arr)
        #for k in range(shipment_arrival_duation_minutes_loop):

In [ ]:
#BD Assgnment Time constraint
for i in two_drop_zones_ULD_Numbers:
    #for j in BreakDownZoneName_list:
        #print(int(i),str(j))
    BreakDownZoneName_list1 = list(df_merge1[df_merge1.unique_id==i]['BreakDownZoneName'])
    bd_NML_list = [k for k in BreakDownZoneName_list1 if 'NML' in k]
    new_check = df_merge1[(df_merge1.unique_id==i)]
    arr = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
    shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
    #arrival_time_each_uld = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
    transport_time_each_bd_zone = list(new_check['m'].unique())[0]
    if(len(bd_NML_list)!=0):
        model.addConstr(quicksum((c-df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['m'])*x[i,b,c] for b in bd_NML_list for c in range((min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((arr-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))), (min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((arr-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))) +shipment_arrival_duation_minutes_loop, df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m'])) >= arr)
        #for k in range(shipment_arrival_duation_minutes_loop):

In [ ]:
for j in BreakDownZoneName_list:
        #print(int(i),str(j))
    new_check = df_merge1[df_merge1.BreakDownZoneName==j]
    min_time = min(list(new_check['Shipment_arrival_time_minutes'].unique()))
    max_time = max(list(new_check['Shipment_departure_time_minutes'].unique()))
    uld_break = list(new_check['unique_id'].unique())
    #shipment_arrival_time_minutes_loop = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
    #shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
    handling_minutes = list(new_check['handling_m'].unique())[0]
    #print(handling_minutes)
    cap = list(new_check['NumberOfWorkstations'].unique())[0]
    for t in range(min_time, max_time-handling_minutes-1, handling_minutes):
        #print(t)
        uld_list1=[]
        for i in uld_break:
            if(t in range((min_time + (math.ceil((new_check[new_check.unique_id==i].iloc[0]['Shipment_arrival_time_minutes']-min_time)/df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m'])*df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m'])), (min_time + (math.ceil((new_check[new_check.unique_id==i].iloc[0]['Shipment_arrival_time_minutes']-min_time)/df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m'])*df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']))+shipment_arrival_duation_minutes_loop, df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m'])):#new_check[new_check.ArrivalULDNumber==i].iloc[0]['Shipment_departure_time_minutes'])): 
                uld_list1.append(i)
                #print(t,uld_list1)
        if(len(uld_list1)!=0):
            model.addConstr(quicksum(x[a,j,t] for a in uld_list1) <= cap)

In [ ]:
##########################################################################################################################################################################################
##########################################################################################################################################################################################

In [ ]:
#BU Constraints

In [ ]:
for k in filghts_list:
    shipment_date_list = list(sample_shipments_bu[sample_shipments_bu.FlightNumberDeparture==k]['departure_date'].unique())
    for d in shipment_date_list:
        shipment_list = list(sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k) & (sample_shipments_bu.departure_date==d)]['shipment_ID'])
        ws_list = sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k) & (sample_shipments_bu.departure_date==d)]['Count'].unique()[0]
        shipment_departure_time_minutes_loop = sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k) & (sample_shipments_bu.departure_date==d)]['Shipment_departure_time_minutes'].unique()[0]
    #print(shipment_list)
        for l in shipment_list:
        #print(int(i),str(j))
            if((len(shipment_list)!=0)):
                model.addConstr(quicksum(z[l,m,t] for m in range(1, ws_list+1) for t in range(shipment_departure_time_minutes_loop-60, shipment_departure_time_minutes_loop))==1)

In [ ]:
for k in filghts_list:
    shipment_date_list = list(sample_shipments_bu[sample_shipments_bu.FlightNumberDeparture==k]['departure_date'].unique())
    for d in shipment_date_list:
        shipment_list = list(sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k) & (sample_shipments_bu.departure_date==d)]['shipment_ID'])
        ws_list = sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k) & (sample_shipments_bu.departure_date==d)]['Count'].unique()[0]
        shipment_departure_time_minutes_loop = sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k) & (sample_shipments_bu.departure_date==d)]['Shipment_departure_time_minutes'].unique()[0]
        bu_handling_time = sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k) & (sample_shipments_bu.departure_date==d)]['handling_ULD_m'].unique()[0]
        #print(bu_handling_time.dtype)
    #print(shipment_list)
        for m in range(1, ws_list+1):
            for t in range(shipment_departure_time_minutes_loop-60, shipment_departure_time_minutes_loop-bu_handling_time):
                if((len(shipment_list)!=0)):
                    model.addConstr(400*q[k,m,t] - quicksum(sample_shipments_bu[sample_shipments_bu.shipment_ID==l].iloc[0]['TotalWeight']*z[l,m,t] for l in shipment_list) 
                                >= quicksum(sample_shipments_bu[sample_shipments_bu.shipment_ID==l].iloc[0]['TotalWeight']*z[l,m,tau] for l in shipment_list for tau in range(t+1,t+bu_handling_time)))    

In [ ]:
for k in filghts_list:
    shipment_date_list = list(sample_shipments_bu[sample_shipments_bu.FlightNumberDeparture==k]['departure_date'].unique())
    for d in shipment_date_list:
        shipment_list = list(sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k)  & (sample_shipments_bu.departure_date==d)]['shipment_ID'])
        ws_list = sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k)   & (sample_shipments_bu.departure_date==d)]['Count'].unique()[0]
        shipment_departure_time_minutes_loop = sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k)  & (sample_shipments_bu.departure_date==d)]['Shipment_departure_time_minutes'].unique()[0]
        bu_handling_time = sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k)  & (sample_shipments_bu.departure_date==d)]['handling_ULD_m'].unique()[0]
        if((len(shipment_list)!=0)):
            model.addConstr(quicksum(q[k,m,t] for m in range(1, ws_list+1) for t in range(shipment_departure_time_minutes_loop-60, shipment_departure_time_minutes_loop)) <= (quicksum(sample_shipments_bu[sample_shipments_bu.shipment_ID==l].iloc[0]['TotalWeight']*z[l,m,t] for l in shipment_list for t in range(shipment_departure_time_minutes_loop-60,shipment_departure_time_minutes_loop) for m in range(1, ws_list+1))/400) + 1)

In [ ]:
for k in filghts_list:
    shipment_date_list = list(sample_shipments_bu[sample_shipments_bu.FlightNumberDeparture==k]['departure_date'].unique())
    for d in shipment_date_list:
        bu_zone_for_flight = sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k) & (sample_shipments_bu.departure_date==d)]['Name_x'].unique()[0]
        othe_flights_df = sample_shipments_bu[(sample_shipments_bu.Name_x==bu_zone_for_flight) & (sample_shipments_bu.departure_date==d)]
        ws_list = sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k) & (sample_shipments_bu.departure_date==d)]['Count'].unique()[0]
        shipment_departure_time_minutes_loop = sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k) & (sample_shipments_bu.departure_date==d)]['Shipment_departure_time_minutes'].unique()[0]
        other_flights = list(othe_flights_df[(othe_flights_df['FlightNumberDeparture'].isin([k]) == False) & (othe_flights_df.departure_date==d)]['FlightNumberDeparture'].unique())
        bu_handling_time = sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==k) & (sample_shipments_bu.departure_date==d)]['handling_ULD_m'].unique()[0]
        #print(k, "----",filghts_list,"-------", other_flights) 
        #print(shipment_list)
        for m in range(1, ws_list+1):
        #print(int(i),str(j))
            for t in range(shipment_departure_time_minutes_loop-60, shipment_departure_time_minutes_loop - bu_handling_time):
                other_list  =[]
                for i in other_flights:
                    if(t in range((sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==i)].iloc[0]['Shipment_departure_time_minutes']-60),sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==i)].iloc[0]['Shipment_departure_time_minutes'])): 
                        other_list.append(i)
                #print(k, "----",other_list)
                if(len(other_list)!=0):
                    model.addConstr(1-q[k,m,t] >= quicksum(q[p,m,ta] for p in other_list for ta in range(t,min(t+bu_handling_time,(sample_shipments_bu[(sample_shipments_bu.FlightNumberDeparture==p)].iloc[0]['Shipment_departure_time_minutes'])))))

In [ ]:
for i in (list(set(uld_uniqueid_list) - set(two_drop_zones_ULD_Numbers))):
    BreakDownZoneName_list1 = list(df_merge1[df_merge1.unique_id==i]['BreakDownZoneName'])
    shipment_date_list = list(sample_shipments_bu[sample_shipments_bu.unique_id==i]['departure_date'].unique())
    for d in shipment_date_list:
        shipment_list = list(sample_shipments_bu[(sample_shipments_bu.unique_id==i) & (sample_shipments_bu.departure_date==d)]['shipment_ID'])
        #print(shipment_list)
        for l in shipment_list:
            number_of_ws = sample_shipments_bu[(sample_shipments_bu.unique_id==i) & (sample_shipments_bu.shipment_ID==l)]['Count'].unique()[0]
            shipment_departure_time_minutes_loop = sample_shipments_bu[(sample_shipments_bu.unique_id==i) & (sample_shipments_bu.shipment_ID==l)]['Shipment_departure_time_minutes'].unique()[0]
            #shipment_arrival_time_minutes_loop = sample_shipments_bu[(sample_shipments_bu.unique_id==i) & (sample_shipments_bu.shipment_ID==l)]['Shipment_arrival_time_minutes'].unique()[0]
            new_check = df_merge1[(df_merge1.unique_id==i)]
            shipment_arrival_time_minutes_loop = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
            if((len(shipment_list)!=0)):
                model.addConstr(quicksum(t*z[l,m,t] for m in range(1, ws_list+1) for t in range(shipment_departure_time_minutes_loop-60, shipment_departure_time_minutes_loop)) >=
                            quicksum((k + df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m'] + 
                                      df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['m'])*x[i,b,k] for b in BreakDownZoneName_list1 for k in range((min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((shipment_arrival_time_minutes_loop-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))), (min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((shipment_arrival_time_minutes_loop-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))) +shipment_arrival_duation_minutes_loop, df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m'])), name="Constraint 9")

In [ ]:
for i in (two_drop_zones_ULD_Numbers):
    BreakDownZoneName_list1 = list(df_merge1[df_merge1.unique_id==i]['BreakDownZoneName'])
    bd_NRML_list = [k for k in BreakDownZoneName_list1 if 'NRML' in k]
    shipment_date_list = list(sample_shipments_bu[sample_shipments_bu.unique_id==i]['departure_date'].unique())
    for d in shipment_date_list:
        shipment_list = list(sample_shipments_bu[(sample_shipments_bu.unique_id==i) & (sample_shipments_bu.departure_date==d)]['shipment_ID'])
        #print(shipment_list)
        for l in shipment_list:
            number_of_ws = sample_shipments_bu[(sample_shipments_bu.unique_id==i) & (sample_shipments_bu.shipment_ID==l)]['Count'].unique()[0]
            shipment_departure_time_minutes_loop = sample_shipments_bu[(sample_shipments_bu.unique_id==i) & (sample_shipments_bu.shipment_ID==l)]['Shipment_departure_time_minutes'].unique()[0]
            #shipment_arrival_time_minutes_loop = sample_shipments_bu[(sample_shipments_bu.unique_id==i) & (sample_shipments_bu.shipment_ID==l)]['Shipment_arrival_time_minutes'].unique()[0]
            new_check = df_merge1[(df_merge1.unique_id==i)]
            shipment_arrival_time_minutes_loop = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
            if((len(shipment_list)!=0)):
                model.addConstr(quicksum(t*z[l,m,t] for m in range(1, ws_list+1) for t in range(shipment_departure_time_minutes_loop-60, shipment_departure_time_minutes_loop)) >=
                            quicksum((k + df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m'] + 
                                      df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['m'])*x[i,b,k] for b in bd_NRML_list for k in range((min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((shipment_arrival_time_minutes_loop-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))), (min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((shipment_arrival_time_minutes_loop-min(list(df_merge1[df_merge1.BreakDownZoneName==b]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m']))) +shipment_arrival_duation_minutes_loop, df_merge1[df_merge1.BreakDownZoneName==b].iloc[0]['handling_m'])), name="Constraint Others")

In [ ]:
obj = 0
for i in ArrivalULDNumber_list:
    shipment_date_list = list(sample_shipments_bu[sample_shipments_bu.ArrivalULDNumber==i]['departure_date'].unique())
    for d in shipment_date_list:
        shipment_list = list(sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.departure_date==d)]['shipment_ID'])
    #print(shipment_list)
        for l in shipment_list:
        #print(int(i),str(j))
            number_of_ws = sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.shipment_ID==l)]['Count'].unique()[0]
            shipment_departure_time_minutes_loop = sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.shipment_ID==l)]['Shipment_departure_time_minutes'].unique()[0]
            if((len(shipment_list)!=0)):
                obj += quicksum(t*z[l,m,t] for m in range(1,number_of_ws+1) for t in range(shipment_departure_time_minutes_loop-60, shipment_departure_time_minutes_loop))

In [ ]:
model.setObjective(obj, GRB.MINIMIZE)
model.optimize()

In [ ]:
#import datetime
print(dt.datetime.now())

In [ ]:
model.write('Full_Model_1.lp')

In [ ]:
for i in uld_uniqueid_list:
    BreakDownZoneName_list1 = list(df_merge1[df_merge1.unique_id==i]['BreakDownZoneName'])
    #print(i, "in", BreakDownZoneName_list1)
    for j in BreakDownZoneName_list1:
        #print(int(i),str(j))
        new_check = df_merge1[(df_merge1.unique_id==i) & (df_merge1.BreakDownZoneName==j)]
        shipment_arrival_time_minutes_loop = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
        shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
        for k in range((min(list(df_merge1[df_merge1.BreakDownZoneName==j]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((shipment_arrival_time_minutes_loop-min(list(df_merge1[df_merge1.BreakDownZoneName==j]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']))), (min(list(df_merge1[df_merge1.BreakDownZoneName==j]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((shipment_arrival_time_minutes_loop-min(list(df_merge1[df_merge1.BreakDownZoneName==j]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']))) +shipment_arrival_duation_minutes_loop, df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']):
            if x[i,j,k].x > 0.5:
                print("ULD", i, "assigned to BD", j, "at", k)
                #print(k)
            #if(i == '51228_25671360'):
                #print(x)
                    

In [ ]:
# visualize the optimal assignment
for i in uld_uniqueid_list:
    BreakDownZoneName_list1 = list(df_merge1[df_merge1.unique_id==i]['BreakDownZoneName'])
    for j in BreakDownZoneName_list1:
        #print(int(i),str(j))
        new_check = df_merge1[(df_merge1.unique_id==i) & (df_merge1.BreakDownZoneName==j)]
        shipment_arrival_time_minutes_loop = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
        shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
        for k in range((min(list(df_merge1[df_merge1.BreakDownZoneName==j]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((shipment_arrival_time_minutes_loop-min(list(df_merge1[df_merge1.BreakDownZoneName==j]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']))), (min(list(df_merge1[df_merge1.BreakDownZoneName==j]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((shipment_arrival_time_minutes_loop-min(list(df_merge1[df_merge1.BreakDownZoneName==j]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']))) +shipment_arrival_duation_minutes_loop, df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']):
            if x[i,j,k].x > 0.5:
                #print(k)
                #print("DZ", i, "assigned to BD", j, "at", (shipment_arrival_time_minutes_loop+k))
                plt.plot(k, j, "o")
                #plt.annotate(i, (k,j))
# for j in range(m):    
#     plt.plot(1, j, "o")
# for i in range(n):
#     for j in range(m):
#         if x[i,j].x > 0.5:
#             plt.plot([0,1], [i,j], "k")# todo
plt.show

In [ ]:
for i in ArrivalULDNumber_list:
    shipment_list = list(sample_shipments_bu[sample_shipments_bu.ArrivalULDNumber==i]['shipment_ID'])
    #print(shipment_list)
    for l in shipment_list:
        #print(int(i),str(j))
        number_of_ws = sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.shipment_ID==l)]['Count'].unique()[0]
        shipment_departure_time_minutes_loop = sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.shipment_ID==l)]['Shipment_departure_time_minutes'].unique()[0]
        for m in range(1,number_of_ws+1):
            for t in range(shipment_departure_time_minutes_loop-60, shipment_departure_time_minutes_loop):
                if z[l,m,t].x > 0.5:
                    print("Shipment", l, "assigned to WS", m, "at", t)

In [ ]:
for i in ArrivalULDNumber_list:
    shipment_list = list(sample_shipments_bu[sample_shipments_bu.ArrivalULDNumber==i]['shipment_ID'])
    #print(shipment_list)
    for l in shipment_list:
        #print(int(i),str(j))
        number_of_ws = sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.shipment_ID==l)]['Count'].unique()[0]
        shipment_departure_time_minutes_loop = sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.shipment_ID==l)]['Shipment_departure_time_minutes'].unique()[0]
        bu_zone_name = sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.shipment_ID==l)]['Name_x'].unique()[0]
        flight_number = sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.shipment_ID==l)]['FlightNumberDeparture'].unique()[0]
        for m in range(1,number_of_ws+1):
            for t in range(shipment_departure_time_minutes_loop-60, shipment_departure_time_minutes_loop):
                if z[l,m,t].x > 0.5:
                    k = str(bu_zone_name +"-WS-" + str(m))
                    #print("Shipment", l, "assigned to", bu_zone_name,"WS",m, "at", t)
                    plt.plot(t, k, "o")
                    #plt.annotate(l, (t,k))
plt.show

In [ ]:
#################################################################################################################################
#################################################################################################################################
#################################################################################################################################

In [ ]:
df_final_x = []
for i in uld_uniqueid_list:
    BreakDownZoneName_list1 = list(df_merge1[df_merge1.unique_id==i]['BreakDownZoneName'])
    #print(i, "in", BreakDownZoneName_list1)
    for j in BreakDownZoneName_list1:
        #print(int(i),str(j))
        new_check = df_merge1[(df_merge1.unique_id==i) & (df_merge1.BreakDownZoneName==j)]
        shipment_arrival_time_minutes_loop = list(new_check['Shipment_arrival_time_minutes'].unique())[0]
        shipment_arrival_duation_minutes_loop = list(new_check['one_hour_duration'].unique())[0] #list(new_check['Shipment_duration_minutes'].unique())[0]
        for k in range((min(list(df_merge1[df_merge1.BreakDownZoneName==j]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((shipment_arrival_time_minutes_loop-min(list(df_merge1[df_merge1.BreakDownZoneName==j]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']))), (min(list(df_merge1[df_merge1.BreakDownZoneName==j]['Shipment_arrival_time_minutes'].unique())) + (math.ceil((shipment_arrival_time_minutes_loop-min(list(df_merge1[df_merge1.BreakDownZoneName==j]['Shipment_arrival_time_minutes'].unique())))/(df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']))*(df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']))) +shipment_arrival_duation_minutes_loop, df_merge1[df_merge1.BreakDownZoneName==j].iloc[0]['handling_m']):
            if x[i,j,k].x > 0.5:
                df_final_x.append((i, j, k))


df_final_x1 = pd.DataFrame(df_final_x, columns=('unique_id', 'bd_name', 'assigned_time'))

In [ ]:
df_final_x1.head()

In [ ]:
df_final_x1.to_csv(r'bd_assigned_time_output.csv')

In [ ]:
df_final_z = []
for i in ArrivalULDNumber_list:
    shipment_list = list(sample_shipments_bu[sample_shipments_bu.ArrivalULDNumber==i]['shipment_ID'])
    #print(shipment_list)
    for l in shipment_list:
        #print(int(i),str(j))
        number_of_ws = sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.shipment_ID==l)]['Count'].unique()[0]
        shipment_departure_time_minutes_loop = sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.shipment_ID==l)]['Shipment_departure_time_minutes'].unique()[0]
        bu_zone_name = sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.shipment_ID==l)]['Name_x'].unique()[0]
        flight_number = sample_shipments_bu[(sample_shipments_bu.ArrivalULDNumber==i) & (sample_shipments_bu.shipment_ID==l)]['FlightNumberDeparture'].unique()[0]
        for m in range(1,number_of_ws+1):
            for t in range(shipment_departure_time_minutes_loop-60, shipment_departure_time_minutes_loop):
                if z[l,m,t].x > 0.5:
                    #k = str(bu_zone_name +"-WS-" + str(m))
                    df_final_z.append((l,bu_zone_name ,m, t))

df_final_z = pd.DataFrame(df_final_z, columns=('shipment_id', 'bu_name', 'ws_number', 'buildup_time'))

In [ ]:
df_final_z.head()

In [ ]:
df_final_z.to_csv(r'bu_workstation_time_output.csv')

In [ ]:
sample_shipments.to_csv(r'sample_shipments.csv')
sample_shipments_bu.to_csv(r'sample_shipments_bu.csv')
booking_new.to_csv(r'booking_new.csv')
booking_new_BU.to_csv(r'booking_new_BU.csv')
df_merge1.to_csv(r'df_merge1.csv')
two_drop_zones.to_csv(r'two_drop_zones.csv')
new.to_csv(r'new.csv')
bu_zone_all.to_csv(r'bu_zone_all.csv')
uld_bd_data.to_csv(r'uld_bd_data.csv')
bu_workstation_count.to_csv(r'bu_workstation_count.csv')
uld_bd_data.to_csv(r'uld_bd_data.csv')
dz_ship_data.to_csv(r'dz_ship_data.csv')
dz_to_bd.to_csv(r'dz_to_bd.csv')
multiple_uld_number_and_times.to_csv(r'multiple_uld_number_and_times.csv')
unique_uld_count_by_number.to_csv(r'unique_uld_count_by_number.csv')
flight_pre_processing_all.to_csv(r'flight_pre_processing_all.csv')
flight_default_processing.to_csv(r'flight_default_processing.csv')
uld_number_times.to_csv(r'uld_number_times.csv')
bu_zones_to_flight.to_csv(r'bu_zones_to_flight.csv')
bd_zones.to_csv(r'bd_zones.csv')
bd_capacity.to_csv(r'bd_capacity.csv')

In [ ]:
##############################################################################################